In [8]:
from django.http import HttpResponse
from collections import Counter
import statsmodels.api as sm
import pandas as pd
import numpy as np
import json
import re
import sys
import os

MIDAS_DATA = "MIDAS_unified-latest.txt"
CHEMONTID_DICTIONARY = "CHEMONTID-mapper.json"

In [4]:
def test_path():
    DATA_PATH = os.path.join(
        os.getcwd(),
        "data"
    )


def import_table(
        _path,
        _file,
        index_col=None):
    """Import URL as tab-delimited table
    """

    return pd.read_csv(
        os.path.join(_path, _file),
        sep='\t',
        index_col=index_col,
        low_memory=False
    )


def import_database_str(
        database,
        index_col=None):
    """Import URL as tab-delimited table
    """

    from io import StringIO
    return pd.read_csv(
        StringIO(database),
        sep='\t',
        index_col=index_col,
        low_memory=False
    )


def import_json(
        _path,
        _file):
    """Import URL as JSON-formatted dictionary
    """
    with open(os.path.join(_path, _file)) as json_file:
        data = json.load(json_file)

    return data



def crossref_databases(
        midas_table,
        substructure_dictionary,
        metabolite_reference):
    """Crossreference MIDAS data with the substructure database using the Electrum metabolite mapping reference
    """
    # Use to prevent excessive warning messages
    non_mappers = set()
    non_matchers = set()
    non_hmdb = set()

    midas_table_c = midas_table.copy()

    # Add HMDB ID to MIDAS table
    # Add substructure annotations to MIDAS table
    for index, row in midas_table_c.iterrows():

        if ';' in row['metabolite']:
            metabolite = row['metabolite'].split(';')[0]
        else:
            metabolite = row['metabolite']

        metabolite_simplified = re.sub(r'\W+', '', metabolite).lower()

        if metabolite_simplified in metabolite_reference:
            hmdb_id = metabolite_reference[metabolite_simplified]['hmdb_id']

            if 'hmdb' in str(hmdb_id).lower():

                # Add HMDB ID
                midas_table_c.at[index, 'HMDB_ID'] = str(hmdb_id)

                # Add sub-structure IDs and names
                i = 0
                while i < 12:
                    hmdb_id = 'HMDB' + hmdb_id.replace('HMDB', '').zfill(i)
                    if hmdb_id in substructure_dictionary:
                        midas_table_c.at[index, 'taxonomy_ids'] = (
                            substructure_dictionary[hmdb_id]['taxonomy_ids']
                        )
                        midas_table_c.at[index, 'taxonomy_terms'] = (
                            substructure_dictionary[hmdb_id]['taxonomy_terms']
                        )
                        break
                    else:
                        i += 1
                else:
                    if metabolite not in non_hmdb:
                        print(
                            'Unable to find', hmdb_id,
                            '(', metabolite, ') in substructure database')
                        non_hmdb.add(metabolite)

            else:
                if metabolite not in non_mappers:
                    print("HMDB ID not available for", metabolite)
                    non_mappers.add(metabolite)
        else:
            if metabolite not in non_matchers:
                print('Unable to match', metabolite)
                non_matchers.add(metabolite)

    return midas_table_c


def substructure_enrichment(
        unified_table,
        chemontid_reference,
        TARGET,
        THRESHOLD):
    """Perform enrichment analysis
    """ 
    # Target selection table 
    unified_table_target = unified_table.loc[unified_table['query_protein'] == TARGET]

    print("----")
    print("Input table:")
    print("----")
    print(unified_table_target)
    print("----")

    # Init results table 
    results_table = pd.DataFrame()
    results_table["CHEMONTID"] = unified_table_target["taxonomy_ids"].str.split(';').explode('taxonomy_ids').unique().tolist()
    results_table.index = results_table["CHEMONTID"]
    results_table.index.name = None

    # Generate EXPECTED CHEMONTID distributions

    ### (C) Get expected 
    expected_counter = Counter(unified_table_target["taxonomy_ids"].str.split(';').explode('taxonomy_ids').tolist())
    results_table["C"] = 0
    results_table["C"] = results_table["CHEMONTID"].map(expected_counter).fillna(results_table["C"])

    ### (D) Total number of metabolites in library
    LIBRARY_SIZE = len(unified_table["metabolite"].unique().tolist()) 
    results_table["D"] = LIBRARY_SIZE - results_table["C"]

    # Generate OBSERVED CHEMONTID distributions
    unified_table_threshold = unified_table_target.loc[unified_table_target['q_value'] < THRESHOLD]

    ### (A) Get observed 
    observed_counter = Counter(unified_table_threshold["taxonomy_ids"].str.split(';').explode('taxonomy_ids').tolist())
    results_table["A"] = 0
    results_table["A"] = results_table["CHEMONTID"].map(observed_counter).fillna(results_table["A"])

    ### (B) Total number of observed metabolites
    OBSERVED_COUNT = len(unified_table_threshold["metabolite"].unique().tolist()) 
    results_table["B"] = OBSERVED_COUNT - results_table["A"]

    results_table = results_table[["CHEMONTID", "A", "B", "C", "D"]]


    #Fisher Exact Test. For each row, runs Fisher Exact on 4 columns and outputs final result to new column.
    #Vectorizing the below could speed it up if we still want live p-value updates:
    #https://stackoverflow.com/questions/34947578/how-to-vectorize-fishers-exact-test
    
    # Remove any substructures where there are 0 or 1 counts to prevent weighting downstream FDR
    results_table = results_table.loc[results_table["A"] > 1]

    results_table["Fold_change"] = (results_table["A"] / results_table["B"]) / (results_table["C"] / results_table["D"])

    _arr = results_table[['A', 'B', 'C', 'D']].to_numpy(dtype=np.uint, copy=True)
    _, _, twosided = pvalue_npy(_arr[:, 0], _arr[:, 1], _arr[:, 2], _arr[:, 3])
    results_table["P_value"] = twosided
    results_table = results_table.sort_values(by="P_value")

    _, results_table["FDR"], _, _ = sm.stats.multipletests(
        results_table["P_value"].values,
        alpha=THRESHOLD,
        method="fdr_bh",
        is_sorted=True
    )

    # Add common substructure names to results table 
    results_table["Term"] = results_table["CHEMONTID"].map(chemontid_reference).fillna(results_table["CHEMONTID"])

    return results_table

In [6]:
TARGET = str("CS")
THRESHOLD = float(0.1)

In [9]:
DATA_PATH = os.path.join(
    os.getcwd(),
    "..",
    "..",
    "..",
    "data"
)

In [22]:
unified_table = import_table(
        DATA_PATH,
        "MIDAS-latest.txt",
        index_col=None)

In [23]:
unified_table.head()

,metabolite,query_protein,log2_abundance,log2_abundance_corrected,met_mean,met_sd,p_value,q_value
0,O-Phospho-L-serine,ALDOA,-3.145748,-2.583616,-0.041294,0.134860,2.850000e-79,1.930000e-76
1,2-Deoxycytidine 5-diphosphate,ALDOA,-2.049392,-1.224692,-0.038301,0.122482,3.450000e-22,5.740000e-20
2,Cytidine 5-diphosphate,ALDOA,-2.324957,-1.249463,-0.038824,0.163332,1.240000e-13,1.170000e-11
3,Hippuric acid,ALDOA,-1.049454,-0.622906,0.022553,0.089612,5.900000e-13,5.230000e-11
4,D-3-Phosphoglyceric acid,ALDOA,0.513185,1.065757,-0.012776,0.156700,5.870000e-12,4.700000e-10


In [24]:
metabolite_reference = import_json(
        _path=DATA_PATH,
        _file="metabolites.json")

In [25]:
substructure_dictionary = import_json(
        _path=DATA_PATH,
        _file="CHEMONTID-substructure-dictionary.json")

In [26]:
unified_table_c = crossref_databases(
        midas_table=unified_table,
        substructure_dictionary=substructure_dictionary,
        metabolite_reference=metabolite_reference)

HMDB ID not available for D-3-Phosphoglyceric acid
HMDB ID not available for 5-Phospho-D-ribose 1-diphosphate
HMDB ID not available for D-Glyceraldehyde 3-phosphate
HMDB ID not available for Xanthosine 5-monophosphate
HMDB ID not available for Nicotinamide hypoxanthine dinucleotide
HMDB ID not available for DL-3,4-Dihydroxymandelic acid
HMDB ID not available for D-Erythrono-1,4-lactone
HMDB ID not available for PIP2
HMDB ID not available for 2,3-Dihydroxypropyl hexopyranoside
HMDB ID not available for DL-5-Hydroxylysine
HMDB ID not available for PIP3
HMDB ID not available for Crotonoyl coenzyme A
HMDB ID not available for Ascorbate
HMDB ID not available for DL-3,4-Dihydroxyphenyl glycol
HMDB ID not available for Phosphatidylcholine
HMDB ID not available for Phosphatidylglycerol
HMDB ID not available for 3-Aminopiperidin-2-one
HMDB ID not available for (4R)-4-Hydroxy-L-glutamic acid
HMDB ID not available for Phosphatidic Acid
HMDB ID not available for 6-Deoxy-L-galactonic acid
HMDB ID n

In [27]:
unified_table_c.loc[unified_table_c["query_protein"] == "CS"].head()

,metabolite,query_protein,log2_abundance,log2_abundance_corrected,met_mean,met_sd,p_value,q_value,HMDB_ID,taxonomy_ids,taxonomy_terms
999,Pyruvate,CS,0.496223,0.540246,-0.013880,0.057726,8.050000e-22,1.290000e-19,HMDB0000243,CHEMONTID:0002912;CHEMONTID:0001113;CHEMONTID:...,Alpha-hydroxy ketones;Alpha-keto acids and der...
1000,3-Phosphoadenosine 5-phosphosulfate,CS,0.632338,0.691108,-0.024273,0.154863,3.850000e-06,1.420000e-04,HMDB0001134,CHEMONTID:0002987;CHEMONTID:0000129;CHEMONTID:...,6-aminopurines;Alcohols and polyols;Alkyl phos...
1001,Coenzyme A,CS,3.748431,2.407303,0.101348,0.599103,1.190000e-04,2.984090e-03,HMDB0001423,CHEMONTID:0002987;CHEMONTID:0000129;CHEMONTID:...,6-aminopurines;Alcohols and polyols;Alkyl phos...
1002,3-Hydroxybutyryl coenzyme A,CS,0.117545,0.983857,0.060308,0.335212,5.867083e-03,7.612595e-02,NaN,NaN,NaN
1003,Dephosphocoenzyme A,CS,2.551043,1.538022,-0.048616,0.582845,6.484324e-03,8.218892e-02,HMDB0001373,CHEMONTID:0002987;CHEMONTID:0000129;CHEMONTID:...,6-aminopurines;Alcohols and polyols;Alkyl phos...


In [29]:
unified_table_m

,metabolite,query_protein,log2_abundance,log2_abundance_corrected,met_mean,met_sd,p_value,q_value
0,O-Phospho-L-serine,ALDOA,-3.145748,-2.583616,-0.041294,0.134860,2.850000e-79,1.930000e-76
1,2-Deoxycytidine 5-diphosphate,ALDOA,-2.049392,-1.224692,-0.038301,0.122482,3.450000e-22,5.740000e-20
2,Cytidine 5-diphosphate,ALDOA,-2.324957,-1.249463,-0.038824,0.163332,1.240000e-13,1.170000e-11
3,Hippuric acid,ALDOA,-1.049454,-0.622906,0.022553,0.089612,5.900000e-13,5.230000e-11
4,D-3-Phosphoglyceric acid,ALDOA,0.513185,1.065757,-0.012776,0.156700,5.870000e-12,4.700000e-10
...,...,...,...,...,...,...,...,...
10984,Uridine 5-diphosphoglucose;Uridine 5-diphospho...,TPI1,-0.126697,0.113248,0.062165,0.164413,7.560286e-01,9.999950e-01
10985,Xanthine,TPI1,-0.049788,0.047387,0.081799,0.167427,8.371538e-01,9.999950e-01
10986,Xanthosine,TPI1,-0.064227,0.031926,0.007984,0.058119,6.803818e-01,9.999950e-01
10987,Xanthosine 5-monophosphate,TPI1,-0.059490,-0.001620,0.017586,0.089592,8.302549e-01,9.999950e-01


In [30]:
# Use to prevent excessive warning messages
non_mappers = set()
non_matchers = set()
non_hmdb = set()

In [37]:
unified_table_m = unified_table.copy()

In [38]:
unified_table_m = unified_table_m.loc[unified_table_m["query_protein"] == "CS"].head()

In [39]:
unified_table_m

,metabolite,query_protein,log2_abundance,log2_abundance_corrected,met_mean,met_sd,p_value,q_value
999,Pyruvate,CS,0.496223,0.540246,-0.013880,0.057726,8.050000e-22,1.290000e-19
1000,3-Phosphoadenosine 5-phosphosulfate,CS,0.632338,0.691108,-0.024273,0.154863,3.850000e-06,1.420000e-04
1001,Coenzyme A,CS,3.748431,2.407303,0.101348,0.599103,1.190000e-04,2.984090e-03
1002,3-Hydroxybutyryl coenzyme A,CS,0.117545,0.983857,0.060308,0.335212,5.867083e-03,7.612595e-02
1003,Dephosphocoenzyme A,CS,2.551043,1.538022,-0.048616,0.582845,6.484324e-03,8.218892e-02


In [40]:
# Add HMDB ID to MIDAS table
# Add substructure annotations to MIDAS table
for index, row in unified_table_m.iterrows():

    if ';' in row['metabolite']:
        metabolite = row['metabolite'].split(';')[0]
    else:
        metabolite = row['metabolite']
    
    metabolite_simplified = re.sub(r'\W+', '', metabolite).lower()

    print(metabolite_simplified)
    if metabolite_simplified in metabolite_reference:
        hmdb_id = metabolite_reference[metabolite_simplified]['hmdb_id']
        print(hmdb_id)
        
        if 'hmdb' in str(hmdb_id).lower():

            # Add HMDB ID
            unified_table_m.at[index, 'HMDB_ID'] = str(hmdb_id)

            # Add sub-structure IDs and names
            i = 0
            while i < 12:
                hmdb_id = 'HMDB' + hmdb_id.replace('HMDB', '').zfill(i)
                if hmdb_id in substructure_dictionary:
                    unified_table_m.at[index, 'taxonomy_ids'] = (
                        substructure_dictionary[hmdb_id]['taxonomy_ids']
                    )
                    unified_table_m.at[index, 'taxonomy_terms'] = (
                        substructure_dictionary[hmdb_id]['taxonomy_terms']
                    )
                    break
                else:
                    i += 1
            else:
                if metabolite not in non_hmdb:
                    print(
                        'Unable to find', hmdb_id,
                        '(', metabolite, ') in substructure database')
                    non_hmdb.add(metabolite)

        else:
            if metabolite not in non_mappers:
                print("HMDB ID not available for", metabolite)
                non_mappers.add(metabolite)
    else:
        if metabolite not in non_matchers:
            print('Unable to match', metabolite)
            non_matchers.add(metabolite)
    
    print("---")



pyruvate
HMDB0000243
---
3phosphoadenosine5phosphosulfate
HMDB0001134
---
coenzymea
HMDB0001423
---
3hydroxybutyrylcoenzymea

---
dephosphocoenzymea
HMDB0001373
---


In [34]:
unified_table_m.loc[unified_table_m["query_protein"] == "CS"].head()

,metabolite,query_protein,log2_abundance,log2_abundance_corrected,met_mean,met_sd,p_value,q_value,HMDB_ID,taxonomy_ids,taxonomy_terms
999,Pyruvate,CS,0.496223,0.540246,-0.013880,0.057726,8.050000e-22,1.290000e-19,HMDB0000243,CHEMONTID:0002912;CHEMONTID:0001113;CHEMONTID:...,Alpha-hydroxy ketones;Alpha-keto acids and der...
1000,3-Phosphoadenosine 5-phosphosulfate,CS,0.632338,0.691108,-0.024273,0.154863,3.850000e-06,1.420000e-04,HMDB0001134,CHEMONTID:0002987;CHEMONTID:0000129;CHEMONTID:...,6-aminopurines;Alcohols and polyols;Alkyl phos...
1001,Coenzyme A,CS,3.748431,2.407303,0.101348,0.599103,1.190000e-04,2.984090e-03,HMDB0001423,CHEMONTID:0002987;CHEMONTID:0000129;CHEMONTID:...,6-aminopurines;Alcohols and polyols;Alkyl phos...
1002,3-Hydroxybutyryl coenzyme A,CS,0.117545,0.983857,0.060308,0.335212,5.867083e-03,7.612595e-02,NaN,NaN,NaN
1003,Dephosphocoenzyme A,CS,2.551043,1.538022,-0.048616,0.582845,6.484324e-03,8.218892e-02,HMDB0001373,CHEMONTID:0002987;CHEMONTID:0000129;CHEMONTID:...,6-aminopurines;Alcohols and polyols;Alkyl phos...


In [44]:
metabolite_reference[x]

{'alt_name': '(4R)-4-Hydroxy-L-glutamic acid',
 'chebi_id': '',
 'hmdb_id': '',
 'kegg_id': '',
 'name': '(4R)-4-Hydroxy-L-glutamic acid',
 'smiles': ''}

In [51]:
for x in metabolite_reference.keys():
    if "coenzyme" in metabolite_reference[x]["alt_name"]:
        print(metabolite_reference[x])

{'alt_name': '3-Hydroxy-3-methylglutaryl coenzyme A', 'chebi_id': '', 'hmdb_id': '', 'kegg_id': '', 'name': '3-Hydroxy-3-methylglutaryl coenzyme A', 'smiles': ''}
{'alt_name': '3-Hydroxybutyryl coenzyme A', 'chebi_id': '', 'hmdb_id': '', 'kegg_id': '', 'name': '3-Hydroxybutyryl coenzyme A', 'smiles': ''}
{'alt_name': '3-Hydroxy-3-methylglutaryl coenzyme A', 'chebi_id': '', 'hmdb_id': '', 'kegg_id': '', 'name': '3-Hydroxy-3-methylglutaryl coenzyme A', 'smiles': ''}
{'alt_name': '3-Hydroxybutyryl coenzyme A', 'chebi_id': '', 'hmdb_id': '', 'kegg_id': '', 'name': '3-Hydroxybutyryl coenzyme A', 'smiles': ''}
{'alt_name': 'Butyryl coenzyme A', 'chebi_id': '', 'hmdb_id': '', 'kegg_id': '', 'name': 'Butyryl coenzyme A', 'smiles': ''}
{'alt_name': 'Crotonoyl coenzyme A', 'chebi_id': '', 'hmdb_id': '', 'kegg_id': '', 'name': 'Crotonoyl coenzyme A', 'smiles': ''}
{'alt_name': 'Octanoyl coenzyme A', 'chebi_id': '', 'hmdb_id': '', 'kegg_id': '', 'name': 'Octanoyl coenzyme A', 'smiles': ''}
{'alt_n

In [ ]:
chemontid_reference = import_json(
        _path=DATA_PATH,
        _file=CHEMONTID_DICTIONARY)

In [ ]:
results = substructure_enrichment(
        unified_table=unified_table,
        chemontid_reference=chemontid_reference,
        TARGET=TARGET,
        THRESHOLD=THRESHOLD)

In [ ]:
print("----")
print("Results table:")
print("----")
print(results)
print("----")